# CNN

In [5]:
# ! pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [22]:
''' 
We import the necessary libraries and modules for our code.
We use PyTorch as our deep learning framework, 
nn module for defining neural networks, 
optim module for optimization algorithms, 
and transforms module for data preprocessing.
'''
# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        """
        CNN Architecture: 2 convolutional layers, 1 max pool, 2 fully connected layer, 
        and final fully connected layer to classify 10 different classes
        """
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # ((256-5)/1) +1 = 252
        self.pool = nn.MaxPool2d(2, 2) # 252 * 252 * 6 -> ((252-2)/2) +1 = 126
        self.conv2 = nn.Conv2d(6, 16, 5) # ((126-5)/1) +1 = 122
        self.fc1 = nn.Linear(16 * 61 * 61, 120) # pool : ((122-2)/2) +1 = 61
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        """
        Defines the forward pass of the network.
        """
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 61 * 61)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [1]:
# from google.colab import drive

# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# from zipfile import ZipFile

# with ZipFile("/content/drive/MyDrive/SI_flood_dataset.zip", 'r') as zip:
#   zip.extractall("/content/drive/MyDrive/SI_flood_dataset")

In [23]:
"""
Creates a data transformation pipeline using the transforms.Compose function from the PyTorch library.

"""
transform = transforms.Compose(
    [transforms.Resize((256, 256)), 
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [24]:
import splitfolders
# Load the dataset

splitfolders.ratio('/content/drive/MyDrive/SI_flood_dataset/dataset/classes', # The location of dataset
                   output="/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split", # The output location
                   seed=42, # The number of seed
                   ratio=(.8, .2,), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=True # If you choose to move, turn this into True
                   )

Copying files: 0 files [00:00, ? files/s]


In [25]:

train_dataSet = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/train', transform=transform) 
test_dataSet = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/val', transform=transform) 


In [26]:
# Data loaders
train_dataset_loader = torch.utils.data.DataLoader(
    train_dataSet, batch_size=4, shuffle=True, num_workers=2
)

test_dataset_loader = torch.utils.data.DataLoader(
    test_dataSet, batch_size=4, shuffle=False, num_workers=2
)

In [27]:
# Define the loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 

In [28]:
# Train the network
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(train_dataset_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in train_dataset_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Finished Training
Accuracy of the network on the 10000 test images: 100 %
